# KLD CUSIP Correction

<div class="alert alert-block alert-warning">
<b>Warning:</b> KLD does not provide CUSIP before 1994. Also, some CUSIP digits are wrongly shifted by omitting one or two zeros in front. When backfill also set '0' and '#N/A' to NA first.
</div>

## Obtain data

In [1]:
import wrds

In [2]:
db = wrds.Connection()

Loading library list...
Done


In [3]:
kld = db.get_table(library='kld', table='history', columns=['companyname', 'cusip', 'ticker', 'companyid', 'year', 'domicile'])
kld.shape

(53168, 6)

In [8]:
kld.sort_values(['year', 'ticker'], inplace=True);

For example, look at the CUSIP of Apple. CUSIP before 1995 is missing and some CUSIP-8-digit wrongly recorded as "37833100" instead of "03783310"!

In [10]:
kld[kld['ticker']=='AAPL']

,companyname,cusip,ticker,companyid,year,domicile
1632,"Apple Computer, Inc.",None,AAPL,None,1991.0,None
1633,"Apple Computer, Inc.",None,AAPL,None,1992.0,None
1634,"Apple Computer, Inc.",None,AAPL,None,1993.0,None
1635,"Apple Computer, Inc.",None,AAPL,None,1994.0,None
1636,"Apple Computer, Inc.",03783310,AAPL,None,1995.0,None
1637,"Apple Computer, Inc.",03783310,AAPL,None,1996.0,None
1638,"Apple Computer, Inc.",03783310,AAPL,None,1997.0,None
1640,"Apple Computer, Inc.",03783310,AAPL,None,1998.0,None
1641,"Apple Computer, Inc.",03783310,AAPL,None,1999.0,None
1642,"Apple Computer, Inc.",37833100,AAPL,None,2000.0,None


In [11]:
kld['0cusip7']='0' + kld['cusip'].str[0:7]
kld['00cusip6']= '00' + kld['cusip'].str[0:6]
kld['000cusip5']= '000' + kld['cusip'].str[0:5]

In [12]:
# checked that cusip8 give the same list as cusip7 if take the first 7 digits
lst111=kld['cusip'].unique()
lst222=kld['0cusip7'].unique()
lst8 = sorted(list(set(lst111) & set(lst222)))
len(lst8)

274

In [13]:
lst1111=kld['cusip'].unique()
lst2222=kld['00cusip6'].unique()
lst88 = sorted(list(set(lst1111) & set(lst2222)))
len(lst88)

59

In [14]:
lst11111=kld['cusip'].unique()
lst22222=kld['000cusip5'].unique()
lst888 = sorted(list(set(lst11111) & set(lst22222)))
len(lst888)

5

In [15]:
rcdict0 = {}
for c in lst8:
    if len(set(kld[kld['0cusip7']==c]['cusip']))==1:
        #print(len(set(kld[kld['0cusip7']==c]['cusip']))==1) # check if wrongly shifted cusip-8-digit is unique
        _c = list(kld[kld['0cusip7']==c]['cusip'])[0]
    rcdict0[_c] = c

In [16]:
# two zero added in front
lst88.remove('00030710')
rcdict00 = {}
for c in lst88:
    if len(set(kld[kld['00cusip6']==c]['cusip']))==1:
        #print(len(set(kld[kld['00cusip6']==c]['cusip']))==1) # check if wrongly shifted cusip-8-digit is unique
        _c = list(kld[kld['00cusip6']==c]['cusip'])[0]
    rcdict00[_c] = c

In [17]:
# three zero added in front
lst888.remove('00030710')
rcdict000 = {}
for c in lst888:
    if len(set(kld[kld['000cusip5']==c]['cusip']))==1:
        #print(len(set(kld[kld['00cusip6']==c]['cusip']))==1) # check if wrongly shifted cusip-8-digit is unique
        _c = list(kld[kld['000cusip5']==c]['cusip'])[0]
    rcdict000[_c] = c

In [ ]:
# Some hand maps
cusip6Dict = {"18772103":"01877210",
              "01877230":"01877210",
              "886309":"00088630"} 

In [ ]:
for d in cusipDict.keys():
    print(cusipDict[d])
    print(kld[kld.cusip==d])